In [1]:
stock = 'TSLA'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    313.579987
2018-07-23    303.200012
2018-07-24    297.429993
2018-07-25    308.739990
2018-07-26    306.649994
2018-07-27    297.179993
2018-07-30    290.170013
2018-07-31    298.140015
2018-08-01    300.839996
2018-08-02    349.540009
2018-08-03    348.170013
2018-08-06    341.989990
2018-08-07    379.570007
2018-08-08    370.339996
2018-08-09    352.450012
2018-08-10    355.489990
2018-08-13    356.410004
2018-08-14    347.640015
2018-08-15    338.690002
2018-08-16    335.450012
2018-08-17    305.500000
2018-08-20    308.440002
2018-08-21    321.899994
2018-08-22    321.640015
2018-08-23    320.100006
2018-08-24    322.820007
2018-08-27    319.269989
2018-08-28    311.859985
2018-08-29    305.010010
2018-08-30    303.149994
                 ...    
2018-09-17    294.839996
2018-09-18    284.959991
2018-09-19    299.019989
2018-09-20    298.329987
2018-09-21    299.100006
2018-09-24    299.679993
2018-09-25    300.989990
2018-09-26    309.579987
2018-09-27    307.51

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


275.5379485584971
[243.71696557 257.33985023 267.13770844 288.59828557 320.89693299]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")